# 🎛️ Advanced Decorator Example with the `@decorator` Library

The [`decorator`](https://pypi.org/project/decorator/) module helps you create **signature-preserving** decorators — which is especially useful for decorators that:

- Accept arguments (like `@warn_slow(timelimit=60)`)
- Need to be readable and clean
- Should keep the original function's name, signature, and docstring

Unlike `functools.wraps`, which only preserves metadata, `@decorator` **automatically preserves the full signature** of the decorated function.

---

## 🚀 Installation

```bash
pip install decorator
```

---

## ✅ Example: Tracing Slow Functions with `@warn_slow`

Let’s say you want to log a warning if a function takes too long to run.  
You can define a parameterized decorator using the `@decorator` style like this:

In [1]:
from decorator import decorator
import time
import logging

logging.basicConfig(level=logging.WARNING)

@decorator
def warn_slow(func, timelimit=60, *args, **kwargs):
    t0 = time.time()
    result = func(*args, **kwargs)
    dt = time.time() - t0

    if dt > timelimit:
        logging.warning('%s took %.1f seconds ⏱️ (above limit!)', func.__name__, dt)

    return result


---

### 🧪 Usage

You can use it both with and without parentheses:

In [2]:
@warn_slow  # uses default timelimit = 60
def preprocess():
    time.sleep(2)

@warn_slow(timelimit=1)  # custom threshold
def quick_task():
    time.sleep(1.5)
    
preprocess()
quick_task()

### 🧠 Using `@warn_slow()` with Parentheses

In addition to using `@warn_slow` without parentheses, you can also call the decorator with parentheses — even if you are not passing any arguments:


In [3]:
@warn_slow()
def another_task():
    time.sleep(1.2)
    
another_task()

This works because the `warn_slow` decorator provides a default value for its parameter (`timelimit=60`).  
So calling `@warn_slow()` is essentially the same as using `@warn_slow` — both apply the decorator with default configuration.

This flexibility allows your decorator to be used in two ways:

- `@warn_slow` — simple application with default values
- `@warn_slow()` — explicit application (still using default values unless specified)

> ✅ The `@decorator` library makes this dual usage possible **without requiring any extra boilerplate logic** in your code.

This dual usage is possible because `@decorator` allows the decorator to accept default arguments — and if the default arguments are acceptable, **you can skip the parentheses**.

---



## 🔍 What's Really Going On?

- When you write `@warn_slow`, it works just like `@warn_slow()`
- You’re defining a **decorator function** with the signature:  
  `decorator(func, *args, **kwargs)`
- `func` is the target function
- `*args` and `**kwargs` are the arguments passed to that function when it runs

---

## ✅ Summary

| Feature                        | Supported by `@decorator` |
|-------------------------------|----------------------------|
| Preserves full function signature | ✅                        |
| Supports decorators with parameters | ✅                    |
| Clean syntax & easy to read   | ✅                        |
| Optional parentheses for defaults | ✅                    |

---

> 🧠 **Tip:** Use the `decorator` module when building advanced decorators for production code, libraries, or educational purposes — it simplifies the hard parts and preserves the beauty of your API.

📚 [Documentation & Examples](https://github.com/micheles/decorator/blob/master/docs/documentation.md)


# 🧱 Stacking Decorators in Python

## 💡 Yes, You Can Stack Decorators!

In Python, you can apply **multiple decorators** to the same function by stacking them one after another:

```python
@decorator_one
@decorator_two
def my_func():
    ...
```

This is equivalent to:

```python
my_func = decorator_one(decorator_two(my_func))
```

That means:
- `decorator_two` is applied **first**
- `decorator_one` wraps the result

---

## ⏱️ Adding a Timing Decorator with Rich Output

Now let’s define a second decorator called `@report_time` that:
- Measures the function’s execution time
- Prints it using **rich** (with styled output)


In [4]:
from functools import wraps
from rich.console import Console
import time

console = Console()

def report_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        duration = time.time() - start
        console.print(f"[bold green]✅ Function [cyan]{func.__name__}[/cyan] ran in [yellow]{duration:.2f}[/yellow] seconds.")
        return result
    return wrapper


Now let’s **stack** it with `@report_time`:

---

## 🎯 Final Example with Stacked Decorators



In [5]:
import time

@report_time
@warn_slow  # uses default timelimit = 60
def preprocess():
    time.sleep(2)

@report_time
@warn_slow(timelimit=1)  # custom threshold
def quick_task():
    time.sleep(1.5)

preprocess()
quick_task()

✅ Function preprocess ran in 2.00 seconds.

✅ Function quick_task ran in 1.50 seconds.


With `rich`, the output will be **colorful and easy to read** — especially in terminal or notebook environments.

---

## ✅ Summary

- You can **stack multiple decorators** to layer behavior
- The **order matters** — inner ones run first
- Use `@report_time` and `@warn_slow` together for beautiful and functional monitoring

> Want to turn this into a decorator library for profiling + alerting? Just say the word!


# Some built-in decorator example

## ✅ 1. `@functools.lru_cache`

### 🔍 What it is:
`@functools.lru_cache` is a **memoization decorator** that caches the results of function calls.  
It's especially useful for expensive or recursive functions, like computing Fibonacci numbers.

### 💡 What it does:
- Saves the output of a function call with a specific input.
- If the same input is used again, the cached result is returned instantly.
- `lru` stands for **Least Recently Used**, meaning it discards old entries when cache size is exceeded.

### 🧪 Example:

In [6]:
import functools
import time
from rich.console import Console

console = Console()

# 🧠 Recursive Fibonacci with caching
@functools.lru_cache(maxsize=128)
def fib(n):
    console.print(f"[dim]Computing fib({n})[/dim]")
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)

# ⏱️ Timing wrapper
def run(func, *args, **kwargs):
    start = time.time_ns()
    result = func(*args, **kwargs)
    stop = time.time_ns()
    duration_sec = (stop - start) * 1e-9
    console.print(f"[green]⏱️ {func.__name__}({', '.join(map(str, args))}) took [bold yellow]{duration_sec:.6f}[/bold yellow] seconds[/green]")
    return result

# Clear cache before starting
fib.cache_clear()

# 🔁 First run: heavy computation
console.rule("Run 1: fib(10)")
console.print("Result:", run(fib, 10))

# 🔁 Second run: partial cache reuse
console.rule("Run 2: fib(11)")
console.print("Result:", run(fib, 11))

# ✅ Fully cached
console.rule("Run 3: fib(9)")
console.print("Result:", run(fib, 9))

# 📦 Cache info
console.rule("LRU Cache Info")
console.print(fib.cache_info())

───────────────────────────────────────────────── Run 1: fib(10) ──────────────────────────────────────────────────

Computing fib(10)

Computing fib(9)

Computing fib(8)

Computing fib(7)

...

────────────────────────────────────────────────── Run 3: fib(9) ──────────────────────────────────────────────────

⏱️ fib(9) took 0.000000 seconds

Result: 34

───────────────────────────────────────────────── LRU Cache Info ──────────────────────────────────────────────────

CacheInfo(hits=11, misses=12, maxsize=128, currsize=12)

### ✅ When to use:
- Recursive functions
- Expensive pure functions (same input → same output)
- Avoiding redundant computation

---

## ✅ 2. `@functools.cache` (Python 3.9+)

### 🔍 What it is:
`@functools.cache` is a simplified version of `lru_cache`, added in Python 3.9.  
It behaves exactly like `@lru_cache(maxsize=None)`, meaning it caches **everything forever**.

### 💡 What it does:
- Simple, unlimited caching
- Ideal for functions with limited and repetitive input space

### 🧪 Example:

In [7]:
import functools

@functools.cache
def multiply(x, y):
    print(f"Calculating {x} * {y}")
    return x * y

print(multiply(2, 3))  # First call: calculates
print(multiply(2, 3))  # Second call: cached

Calculating 2 * 3
6
6


### ✅ When to use:
- Lightweight pure functions
- Where memory usage isn’t a concern
- When you don’t need cache eviction (unlike `lru_cache`)

---

## ✅ 3. `@dataclasses.dataclass`

### 🔍 What it is:
`@dataclass` is a class decorator from the `dataclasses` module that **automatically generates boilerplate code** for classes that are mainly used to store data.

### 💡 What it does:
When you decorate a class with `@dataclass`, it auto-generates:
- `__init__()` constructor
- `__repr__()` for nice printing
- `__eq__()` for comparisons
- Optional: ordering methods (`<`, `<=`, etc.)

### 🧪 Example:

In [8]:
from dataclasses import dataclass

@dataclass
class User:
    name: str
    age: int
    email: str

u1 = User("Ali", 25, "ali@example.com")
console.print(u1)

User(name='Ali', age=25, email='ali@example.com')

### ✅ When to use:
- When building lightweight classes for storing structured data
- Instead of writing repetitive `__init__`, `__repr__`, etc.
- For config objects, DTOs, domain models

---

## 🧠 Summary Table

| Decorator           | Purpose                         | When to Use                                       |
|---------------------|----------------------------------|--------------------------------------------------|
| `@lru_cache`        | Cache results with size limit   | Recursive or slow functions                      |
| `@cache`            | Infinite cache (no limit)       | Simple memoization, few unique inputs            |
| `@dataclass`        | Auto-create methods for classes | Replacing boilerplate in data-holding classes    |

---

> 📌 These decorators help you write **cleaner, faster, and more efficient Python code** with less boilerplate.
